In [5]:
import sys
import numpy as np
import cv2


# 비디오 파일 열기
cap = cv2.VideoCapture('PETS2000.avi')

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

In [6]:
# 배경 영상 등록
ret, back = cap.read()

if not ret:
    print('Background image registration failed!')
    sys.exit()

In [7]:
# 배경 영상
back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0,0), 1) # 블러 처리를 통한 노이즈 제거

fback = back.astype(np.float32) #타입 변환 uint8 -> float32

In [8]:
# 비디오 매 프레임 처리
while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(back, (0,0), 1) # 블러 처리를 통한 노이즈 제거

    cv2.accumulateWeighted(gray, fback, 0.005) # 현재 프레임, float배경 영상 가중치
    back = fback.astype(np.uint8)
    
    diff = cv2.absdiff(back, gray) # 처음 입력 이미지 데이터 타입이랑 낮둥에 뺄 이미지 데이터 타입
    _, diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # 레이블링을 이용하여 바운딩 박스 표시
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(diff)

    for i in range(1, cnt):
        x, y, w, h, s = stats[i]

        if s < 100:
            continue

        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)

    cv2.imshow('frame', frame)
    cv2.imshow('diff', diff)
    cv2.imshow('back', back)

    if cv2.waitKey(30) == 27:
        break

In [9]:
cap.release()
cv2.destroyAllWindows()